In [5]:
import pandas as pd
import pickle
from ast import literal_eval
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, cohen_kappa_score, f1_score , confusion_matrix
from nltk.corpus import stopwords
from sklearn.svm import LinearSVC , NuSVC
from germeval import *
from utils import subtask_A_evaluation ,subtask_A_confusion_matrix , subtask_A_classification_report

In [6]:
mlb =  pickle.load(open("tpv_tl_mlb_author_year_without_num.pickle",'rb'))
year_lb = pickle.load(open("tpv_tl_lb_author_year_without_num.pickle",'rb'))
vectorizer =  pickle.load(open("tpv_tl_train_author_vectorizer_without_num.pickle",'rb'))

X_train = pickle.load(open("tpv_tl_train_author_X_train_without_num.pickle", "rb"))
X_test = pickle.load(open("tpv_tl_train_author_X_test_without_num.pickle", "rb"))
y_train = pickle.load( open("tpv_tl_train_author_y_train_without_num.pickle", "rb"))
y_test = pickle.load(open("tpv_tl_train_author_y_test_without_num.pickle", "rb"))
# le_mapping = pickle.load(open("tpv_tl_train_author_le_mapping.pickle", "rb")) # LabelEncoder Mapping

In [7]:
def build_model(clf, X_train,y_train):
    clf.fit(X_train, y_train)
    return clf

## Linear SVC

In [8]:
X_train

<19356x727991 sparse matrix of type '<class 'numpy.float64'>'
	with 2029432 stored elements in COOrdinate format>

In [9]:
X_test

<4840x727991 sparse matrix of type '<class 'numpy.float64'>'
	with 391465 stored elements in COOrdinate format>

In [10]:
%%time
from scipy import sparse

X_full = sparse.vstack((X_train,X_test))
Y_full = np.concatenate((y_train,y_test))

CPU times: user 44.4 ms, sys: 1.22 ms, total: 45.6 ms
Wall time: 44 ms


In [11]:
X_full.shape

(24196, 727991)

In [12]:
y_train.value_counts()

Literatur & Unterhaltung      8318
Kinderbuch & Jugendbuch       3616
Sachbuch                      3048
Ratgeber                      2298
Ganzheitliches Bewusstsein     963
Glaube & Ethik                 802
Architektur & Garten           179
Künste                         132
Name: toplevel, dtype: int64

In [13]:
del X_test
del X_train
del y_train
del y_test
import gc
gc.collect()

94

In [14]:
# from sklearn.grid_search import GridSearchCV

# param_grid = {'C':[1,10,100,1000],'max_iter' : [1000,2000]}
# grid = GridSearchCV(LinearSVC(),param_grid,refit = True, verbose=2)
# grid.fit(X_train.toarray(),y_train)


In [15]:
# del X_full
# del Y_full
# gc.collect()

In [16]:
X_full_array = X_full.tocsr()

In [20]:
from sklearn.model_selection import cross_val_score , cross_val_predict
lsvcclf = LinearSVC(random_state=42,max_iter=3000,verbose=1,class_weight= {'Kinderbuch & Jugendbuch': 1.8, 'Ratgeber': 3, 'Sachbuch': 2,'Glaube & Ethik' : 2 ,'Künste' : 6,'Architektur & Garten' : 6 })
scores = cross_val_predict(lsvcclf,X_full,y=Y_full, cv=2)
scores

[LibLinear][LibLinear]

array(['Glaube & Ethik', 'Sachbuch', 'Ratgeber', ...,
       'Literatur & Unterhaltung', 'Literatur & Unterhaltung',
       'Literatur & Unterhaltung'], dtype=object)

In [45]:
from sklearn.model_selection import StratifiedKFold
fold_classes = []
probas = []
cv = StratifiedKFold(n_splits=4, random_state=42)
for train_index, test_index in cv.split(X_full_array,Y_full):
    print("Train Index: ", train_index, "\n")
    print("Test Index: ", len(test_index))
    lsvcclf = LinearSVC(random_state=42,max_iter=3000,verbose=1,class_weight= {'Kinderbuch & Jugendbuch': 1.8, 'Ratgeber': 3, 'Sachbuch': 2,'Glaube & Ethik' : 2 ,'Künste' : 6,'Architektur & Garten' : 6 })
    X_train, X_test, y_train, y_test = X_full_array[train_index,:727990], X_full_array[test_index,:727990], Y_full[train_index], Y_full[test_index]
    lsvcclf.fit(X_train, y_train)
    isbn_probas = list(zip(X_full_array[test_index,-1].toarray().astype(int),lsvcclf._predict_proba_lr(X_test)))
    probas.append(isbn_probas)
    fold_classes.append(lsvcclf.classes_)

Train Index:  [ 5772  5830  5854 ... 24193 24194 24195] 

Test Index:  6052
[LibLinear]Train Index:  [    0     1     2 ... 24193 24194 24195] 

Test Index:  6051
[LibLinear]Train Index:  [    0     1     2 ... 24193 24194 24195] 

Test Index:  6047
[LibLinear]Train Index:  [    0     1     2 ... 18318 18325 18407] 

Test Index:  6046
[LibLinear]

In [46]:
fold_classes

[array(['Architektur & Garten', 'Ganzheitliches Bewusstsein',
        'Glaube & Ethik', 'Kinderbuch & Jugendbuch', 'Künste',
        'Literatur & Unterhaltung', 'Ratgeber', 'Sachbuch'], dtype=object),
 array(['Architektur & Garten', 'Ganzheitliches Bewusstsein',
        'Glaube & Ethik', 'Kinderbuch & Jugendbuch', 'Künste',
        'Literatur & Unterhaltung', 'Ratgeber', 'Sachbuch'], dtype=object),
 array(['Architektur & Garten', 'Ganzheitliches Bewusstsein',
        'Glaube & Ethik', 'Kinderbuch & Jugendbuch', 'Künste',
        'Literatur & Unterhaltung', 'Ratgeber', 'Sachbuch'], dtype=object),
 array(['Architektur & Garten', 'Ganzheitliches Bewusstsein',
        'Glaube & Ethik', 'Kinderbuch & Jugendbuch', 'Künste',
        'Literatur & Unterhaltung', 'Ratgeber', 'Sachbuch'], dtype=object)]

In [57]:
all_probas = np.concatenate(probas)

In [66]:
all_isbns = [i[0][0] for i in all_probas]

In [71]:
classes = [ "_".join(i.split(" "))  for i in  ['Architektur & Garten', 'Ganzheitliches Bewusstsein',
        'Glaube & Ethik', 'Kinderbuch & Jugendbuch', 'Künste',
        'Literatur & Unterhaltung', 'Ratgeber', 'Sachbuch']]

prob_dict = {}

for idx,ele in enumerate(classes):
    prob_dict[ele] = [i[1][idx] for i in all_probas]

In [73]:
prob_dict["isbn"] = all_isbns

In [74]:
final_df = pd.DataFrame(prob_dict)

In [76]:
final_df.head(4)

,Architektur_&_Garten,Ganzheitliches_Bewusstsein,Glaube_&_Ethik,Kinderbuch_&_Jugendbuch,Künste,Literatur_&_Unterhaltung,Ratgeber,Sachbuch,isbn
0,0.106866,0.112087,0.158457,0.096202,0.101156,0.136062,0.132551,0.156619,9783466370498
1,0.109238,0.108080,0.112470,0.104316,0.114177,0.133175,0.112870,0.205674,9783791383057
2,0.106624,0.104510,0.110698,0.120817,0.099058,0.152557,0.175001,0.130735,9783641034382
3,0.100739,0.092295,0.105038,0.093231,0.099225,0.113389,0.277256,0.118827,9783641094867


In [78]:
final_df.to_csv("predict_proba.csv",index=False)

# JUNK COPIED 

In [30]:
X_full_array[[5772,5830,5854],:727990]

<3x727991 sparse matrix of type '<class 'numpy.float64'>'
	with 361 stored elements in Compressed Sparse Row format>

In [31]:
np.concatenate(X_full_array[[5772,5830,5854],-1].toarray()).astype(int)

array([9783791356303, 9783453703421, 9783466306008])

In [48]:
X_full_array.shape[1]

727990

In [21]:
probas

array([0.10686607, 0.11208715, 0.15845702, 0.09620181, 0.10115622,
       0.13606174, 0.13255078, 0.15661921])

In [1]:
%%time
# data = X_train.toarray(), X_test.toarray(), y_train, y_test
lsvcclf = LinearSVC(random_state=42,max_iter=3000,verbose=1,class_weight= {'Kinderbuch & Jugendbuch': 1.8, 'Ratgeber': 3, 'Sachbuch': 2,'Glaube & Ethik' : 2 ,'Künste' : 6,'Architektur & Garten' : 6 })
clf = build_model(lsvcclf,X_full,Y_full)

NameError: name 'LinearSVC' is not defined

## Plotting important features

In [54]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt

def plot_coefficients(classifier, vectorizer, top_features=1):
    coef = classifier.coef_
    top_positive_coefficients = np.argsort(coef)[-top_features:]
    top_negative_coefficients = np.argsort(coef)[:top_features]
    top_coefficients = np.hstack([top_negative_coefficients, top_positive_coefficients])
    plt.figure(figsize=(15, 5))
    co = coef[top_coefficients]
    colors = ["red" if c < 0 else "blue" for c in co]
    plt.bar(np.arange(2 * top_features), co, color=colors)
    feature_names = np.array(vectorizer.get_feature_names())
    plt.xticks(np.arange(1, 1 + 2 * top_features), feature_names[top_coefficients], rotation=60, ha="right")
    plt.show()
    
plot_coefficients(lsvcclf, vectorizer.get_feature_names())

MemoryError: 

In [ ]:
a = np.argsort(lsvcclf.coef_.ravel())[:20].tolist()

In [ ]:
b = lsvcclf.coef_.ravel()[a]

In [ ]:
b

In [ ]:
c = np.argsort(lsvcclf.coef_.ravel())[-20:].tolist()

In [ ]:
d = lsvcclf.coef_.ravel()[c]

In [ ]:
d

In [ ]:
len(lsvcclf.coef_)

In [ ]:
len(lsvcclf.coef_[0])

In [ ]:
plt.figure(figsize=(15, 5))
co = np.hstack((b,d))
top_coefficients = np.hstack((a,c))
print(top_coefficients)
len()
print(co)
colors = ["red" if c < 0 else "blue" for c in co]
print(len(co))
plt.bar(np.arange(2 * 20), co, color=colors)
feature_names = np.array(vectorizer.get_feature_names())
plt.xticks(np.arange(1, 1 + 2 * 20), feature_names[top_coefficients], rotation=60, ha="right")
plt.show()

# Lets get some metrics

In [ ]:
set(y_test.values.tolist())

In [ ]:
class_weight = {"Kinderbuch & Jugendbuch":1.8}

In [12]:
%%time
import gc
# data = X_train.toarray(), X_test.toarray(), y_train, y_test
lsvcclf = LinearSVC(random_state=42,verbose=1,class_weight=class_weight)
clf = build_model(lsvcclf,X_train,y_train)
y_pred = clf.predict(X_test)
push_metrics(y_pred,y_test,"linearSVC : " + str(class_weight))

[LibLinear]

CPU times: user 3.46 s, sys: 139 ms, total: 3.6 s
Wall time: 28.1 s


In [ ]:
%matplotlib inline
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
array = confusion_matrix(y_pred,y_test.values)
np.fill_diagonal(array,0)
df_cm = pd.DataFrame(array, index = list(le_mapping.keys()),
                  columns = list(le_mapping.keys()))
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True,fmt=".5g")

In [9]:
# %%time
# from sklearn.linear_model import SGDClassifier
# # data = X_train.toarray(), X_test.toarray(), y_train, y_test
# lsvcclf = SGDClassifier(random_state=42,verbose=1)
# clf = build_model(lsvcclf,X_train,y_train)
# y_pred = clf.predict(X_test)
# push_metrics(y_pred,y_test,"SGDC")

### Run the predictions on valiation for submission

In [13]:
validation_data = pd.read_csv("../../data/labeled_validation_data.csv")

In [14]:
def combine_title_desc(row):
    """Combines the Title and Description fields in the given row
    and returns the combined result"""
    return str(row["title"]) + " " + str(row["description"])
    #return str(row["title"]) + " " + str(row["description"]) + " " + str(row["author"])

In [15]:
validation_tf_idf = vectorizer.transform(validation_data.apply(lambda row: combine_title_desc(row),axis=1))

In [16]:
validation_data["authors"] = validation_data["author"].apply(lambda x:[ i.strip() for i in str(x).split(",")])

In [17]:
validation_data["published_date_parsed"] = pd.to_datetime(validation_data["published_date"],infer_datetime_format=True)

In [18]:
validation_data["year"] = validation_data["published_date_parsed"].apply(lambda x: x.year)

In [19]:
validation_tf_idf.shape

(2079, 638284)

In [20]:
validation_data["authors"].values

array([list(['Hans-Martin Lübking']), list(['Cate Tiernan']),
       list(['Titus Müller']), ..., list(['Arthur C. Clarke']),
       list(['Paul Levine']), list(['Robert Greene'])], dtype=object)

In [21]:
pickle.dump(validation_data["authors"], open("validation_authors", "wb"))

In [22]:
from scipy import sparse

validation_author = mlb.transform(validation_data["authors"].values)
validation_year = year_lb.transform(validation_data["year"].values)
validation_all = sparse.hstack((validation_tf_idf,sparse.csr_matrix(validation_author),sparse.csr_matrix(validation_year)))

In [23]:
predictions = clf._predict_proba_lr(validation_all)

In [24]:
del X_full
del Y_full

In [25]:
import gc
gc.collect()

14

In [26]:
class_counts = pd.read_csv("class_count_predictions_from_svc_validation_set.csv",header=None)

In [27]:
def getClasses(class_probs,class_count):
    arr = np.array(class_probs)
    return arr.argsort()[-1 * class_count:][::-1]

In [28]:
y_prob_list = [i.tolist() for i in predictions]

In [29]:
validation_data["class_probs"] = y_prob_list

In [30]:
validation_data["class_count"] = class_counts.values

In [31]:
# validation_data["predictions"]  = 
validation_data["class_predictions_e"] = validation_data.apply(lambda x:getClasses(x["class_probs"],x["class_count"]),axis=1)

In [32]:
le_mapping_inv = {v: k for k, v in le_mapping.items()}

def reverseMap(number):
    return le_mapping_inv[number]

In [33]:
validation_data["class_predictions"] = validation_data["class_predictions_e"].apply(lambda x: [reverseMap(i) for i in x])

In [34]:
def formatOutput(isbn,class_predictions):
    classes = ["\t"+str(i) for i in class_predictions]
    return str(isbn) + "".join(classes) + "\n"

In [35]:
validation_data["class_predictions_top_level"] = validation_data["class_predictions"].apply(lambda cat_paths: [x.split(" > ")[0] for x in cat_paths])

In [36]:
validation_data["categories"] = validation_data["categories"].apply(lambda x : literal_eval(x))

In [37]:
validation_data["actual_toplevel"] = validation_data["categories"].apply(lambda x:list(set([i.split(" > ")[0] for i in x]))).values

In [38]:
validation_data.head(2)

,Unnamed: 0,title,description,categories,author,published_date,isbn,authors,published_date_parsed,year,class_probs,class_count,class_predictions_e,class_predictions,class_predictions_top_level,actual_toplevel
0,14548,Selbstständig werden im Glauben,Die Konfirmandenzeit wird für Jugendliche beso...,[Glaube & Ethik > Gemeindearbeit > Konfirmatio...,Hans-Martin Lübking,2013-07-22,9783579074030,[Hans-Martin Lübking],2013-07-22,2013,"[0.10417449185542187, 0.09689940202485316, 0.2...",1,[2],[Glaube & Ethik],[Glaube & Ethik],[Glaube & Ethik]
1,14549,Das Buch der Schatten - Schwarze Seelen,Als Morgan von einer prophetischen Vision heim...,"[Kinderbuch & Jugendbuch > Echtes Leben, Reali...",Cate Tiernan,2013-06-10,9783570380093,[Cate Tiernan],2013-06-10,2013,"[0.10656294486286495, 0.10047678792001151, 0.1...",1,[3],[Kinderbuch & Jugendbuch],[Kinderbuch & Jugendbuch],[Kinderbuch & Jugendbuch]


In [39]:
y_true = validation_data["actual_toplevel"].values
y_pred = validation_data["class_predictions_top_level"]
subtask_A_evaluation(y_true,y_pred)

[0.8340807174887892,
 0.8736496007515265,
 0.8534067446662079,
 0.8186628186628186]

In [40]:
y_true = validation_data["actual_toplevel"].values
y_pred = validation_data["class_predictions_top_level"]
subtask_A_confusion_matrix(y_true,y_pred)

array([[[2061,    1],
        [  10,    7]],

       [[1961,    6],
        [  38,   74]],

       [[1983,    6],
        [  27,   63]],

       [[1778,   13],
        [  76,  212]],

       [[2060,    1],
        [  10,    8]],

       [[ 833,  139],
        [  41, 1066]],

       [[1770,   49],
        [  62,  198]],

       [[1687,   54],
        [ 106,  232]]])

In [41]:
y_true = validation_data["actual_toplevel"].values
y_pred = validation_data["class_predictions_top_level"]
print(subtask_A_classification_report(y_true,y_pred))

                            precision    recall  f1-score   support

      Architektur & Garten       0.88      0.41      0.56        17
Ganzheitliches Bewusstsein       0.93      0.66      0.77       112
            Glaube & Ethik       0.91      0.70      0.79        90
   Kinderbuch & Jugendbuch       0.94      0.74      0.83       288
                    Künste       0.89      0.44      0.59        18
  Literatur & Unterhaltung       0.88      0.96      0.92      1107
                  Ratgeber       0.80      0.76      0.78       260
                  Sachbuch       0.81      0.69      0.74       338

                 micro avg       0.87      0.83      0.85      2230
                 macro avg       0.88      0.67      0.75      2230
              weighted avg       0.87      0.83      0.85      2230
               samples avg       0.88      0.85      0.86      2230



In [84]:
y_true

array([list(['Glaube & Ethik']), list(['Kinderbuch & Jugendbuch']),
       list(['Literatur & Unterhaltung']), ...,
       list(['Literatur & Unterhaltung']),
       list(['Literatur & Unterhaltung']),
       list(['Sachbuch', 'Ganzheitliches Bewusstsein', 'Ratgeber'])],
      dtype=object)

In [45]:
submissions = validation_data.apply(lambda x:formatOutput(x["isbn"],x["class_predictions_top_level"]),axis=1).values

In [44]:
submissions = np.insert(submissions,0,"subtask_a\n")

In [45]:
submission_format = "".join(submissions)

In [46]:
submission_file = open("../../data/answers_svm_with_class_unsampling_with_author_year_test_set.txt",'w')

In [47]:
submission_file.writelines(submission_format)

## Lets check the confusion matrix (Do not run this , this will take long time and will not yield good results)

In [9]:
y_test.values

array(['Literatur & Unterhaltung', 'Literatur & Unterhaltung', 'Sachbuch',
       ..., 'Ratgeber', 'Kinderbuch & Jugendbuch',
       'Literatur & Unterhaltung'], dtype=object)

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
array = confusion_matrix(y_pred,y_test.values)
np.fill_diagonal(array,0)
df_cm = pd.DataFrame(array, index = list(le_mapping.keys()),
                  columns = list(le_mapping.keys()))
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True,fmt=".5g")


In [ ]:
inv_map = {v: k for k, v in le_mapping.items()}

In [ ]:
y_pred_top = [inv_map[i].split(" > ")[0] for i in y_pred]

In [ ]:
y_test_top = [inv_map[i].split(" > ")[0] for i in y_test]

In [ ]:
confarray = confusion_matrix(y_test_top, y_pred_top,labels=list(set(y_test_top)))
confarray.shape

In [ ]:
conf = pd.DataFrame(confarray,columns=list(set(y_test_top)))
conf.index = list(set(y_test_top))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize = (10,7))
sns.heatmap(conf, annot=True,fmt="0.5g")

In [17]:
accuracy_score(y_pred_top,y_test_top)

0.8235294117647058

In [21]:
confusion_matrix(y_pred_top,y_test_top)

array([[  36,    0,    0,    1,    0,    0,    3,    1],
       [   1,  144,   11,    0,    0,    1,   35,   14],
       [   0,    8,  123,    2,    0,    9,    0,   28],
       [   0,    3,    1,  687,    2,   83,   19,    9],
       [   0,    0,    2,    2,   19,    0,    2,    8],
       [   2,    8,   19,   77,    3, 1697,   22,  123],
       [   3,   46,    4,   14,    2,    5,  363,   59],
       [   0,    2,   13,    5,    3,   35,   57,  417]])

In [2]:
import spacy
nlp = spacy.load('en')

doc = nlp(u"Apples and oranges are similar. Boots and hippos aren't.")

for token in doc:
    print(token, token.lemma, token.lemma_)

Apples 8566208034543834098 apple
and 2283656566040971221 and
oranges 2208928596161743350 orange
are 10382539506755952630 be
similar 18166476740537071113 similar
. 12646065887601541794 .
Boots 9918665227421442029 boot
and 2283656566040971221 and
hippos 6542994350242320795 hippo
are 10382539506755952630 be
n't 447765159362469301 not
. 12646065887601541794 .


In [4]:
lemmas

['duck']

### Analyse the confusions 

In [12]:
test_books = pd.read_csv("tl_test_books.csv")

In [15]:
test_books["y_test"] = y_test.values
test_books["y_pred"] = y_pred

In [23]:
mistakes = test_books[(test_books["y_pred"] == "Literatur & Unterhaltung") & (test_books["y_test"] == "Kinderbuch & Jugendbuch")]

In [24]:
len(mistakes)

94

In [ ]:
mistakes

In [51]:
pred_test = list(zip(y_pred,y_test))
e = enumerate(pred_test)
ids = [i[0] for i in e if i[1] == ("Literatur & Unterhaltung","Kinderbuch & Jugendbuch")]

In [52]:
ids

[7,
 93,
 102,
 286,
 287,
 415,
 416,
 463,
 529,
 539,
 542,
 553,
 574,
 575,
 594,
 638,
 644,
 671,
 756,
 799,
 806,
 904,
 928,
 970,
 995,
 1008,
 1066,
 1068,
 1225,
 1234,
 1291,
 1301,
 1302,
 1369,
 1435,
 1547,
 1648,
 1650,
 1684,
 1694,
 1727,
 1848,
 1849,
 1928,
 1961,
 2023,
 2034,
 2076,
 2091,
 2126,
 2180,
 2188,
 2189,
 2223,
 2334,
 2356,
 2509,
 2543,
 2544,
 2671,
 2734,
 2745,
 2781,
 2811,
 2820,
 2880,
 2963,
 2966,
 3016,
 3057,
 3070,
 3093,
 3103,
 3112,
 3127,
 3296,
 3299,
 3470,
 3536,
 3574,
 3637,
 3734,
 3805,
 3810,
 3829,
 3838,
 3910,
 3925,
 3948,
 3953,
 4092,
 4155,
 4204,
 4231]

In [36]:
pred_test[10:40]

[('Literatur & Unterhaltung', 'Literatur & Unterhaltung'),
 ('Literatur & Unterhaltung', 'Literatur & Unterhaltung'),
 ('Sachbuch', 'Literatur & Unterhaltung'),
 ('Architektur & Garten', 'Architektur & Garten'),
 ('Literatur & Unterhaltung', 'Literatur & Unterhaltung'),
 ('Literatur & Unterhaltung', 'Literatur & Unterhaltung'),
 ('Ratgeber', 'Ratgeber'),
 ('Ganzheitliches Bewusstsein', 'Ganzheitliches Bewusstsein'),
 ('Ganzheitliches Bewusstsein', 'Ganzheitliches Bewusstsein'),
 ('Architektur & Garten', 'Architektur & Garten'),
 ('Sachbuch', 'Sachbuch'),
 ('Ratgeber', 'Ratgeber'),
 ('Literatur & Unterhaltung', 'Literatur & Unterhaltung'),
 ('Literatur & Unterhaltung', 'Literatur & Unterhaltung'),
 ('Kinderbuch & Jugendbuch', 'Kinderbuch & Jugendbuch'),
 ('Kinderbuch & Jugendbuch', 'Kinderbuch & Jugendbuch'),
 ('Literatur & Unterhaltung', 'Literatur & Unterhaltung'),
 ('Literatur & Unterhaltung', 'Literatur & Unterhaltung'),
 ('Sachbuch', 'Sachbuch'),
 ('Kinderbuch & Jugendbuch', 'Kinde